<a href="https://colab.research.google.com/github/ekrombouts/GenCareAI/blob/main/notebooks/100_note_generation/150_CleanUpNursingHome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GenCare AI: Concat and clean data

**Author:** Eva Rombouts  
**Date:** 2024-09-02  
**Version:** 1.1

### Description
This script concatenates the profiles, scenario's, client records and summaries generated in different experiments (each stored as different 'wards') and restructures and cleans up the data.

The result is a set of 4 datasets:
(If the CARE_HOME variable is set to 'Galaxy':)
1. Galaxy_clients
2. Galaxy_scenarios
3. Galaxy_records
4. Galaxy_summaries

In [ ]:
!pip install GenCareAI
from GenCareAI.GenCareAIUtils import GenCareAISetup

setup = GenCareAISetup()

if setup.environment == 'Colab':
        !pip install -q datasets

In [ ]:
import os
import pandas as pd
import ast
from datetime import datetime, timedelta
import random
from datasets import Dataset

In [ ]:
CARE_HOME = 'Galaxy'
wards = ['Horizon', 'Venus', 'Tulip', 'Cosmos']

FN_CLIENTS_DF = setup.get_file_path(f'data/gcai_{CARE_HOME}_clients.csv')
FN_SCENARIOS_DF = setup.get_file_path(f'data/gcai_{CARE_HOME}_scenarios.csv')
FN_RECORDS_DF = setup.get_file_path(f'data/gcai_{CARE_HOME}_records.csv')
FN_SUMMARIES_DF = setup.get_file_path(f'data/gcai_{CARE_HOME}_summaries.csv')

hf_repo_name = "ekrombouts/" + CARE_HOME

In [ ]:
def update_dag_counter(df):
    """
    This function updates the 'dag' column in df such that it maintains a running counter per client and per week.
    The counter starts at 1 and increments by 1 each time the 'dag' value changes. The counter resets to 1 for each new client and month.
    """
    # Add a column to shift 'dag' values by one row within each group of client_id and month
    df['dag_shift'] = df.groupby(['client_id', 'weekno'])['dag'].shift(1)
    # Create a column indicating if 'dag' has changed compared to the previous row
    df['dag_changed'] = (df['dag'] != df['dag_shift']).astype(int)
    # Create a column indicating the start of a new group (client_id and month)
    df['group_changed'] = df.groupby(['client_id', 'weekno']).cumcount() == 0
    
    # Update 'group_changed' to be False if 'dag_shift' is NaN
    # df['group_changed'] = df['group_changed'] & df['dag_shift'].notna()
    # Create the counter ('teller') by cumulatively summing 'dag_changed' within each group and adding 'group_changed'
    df['dag_running'] = df.groupby(['client_id', 'weekno'])['dag_changed'].cumsum() + df['group_changed']
    
    # Remove the temporary columns used for calculations
    # df.drop(columns=['dag_shift', 'dag_changed', 'group_changed'], inplace=True)
    
    return df

In [ ]:
def concatenate_files(wards, file_prefix):
    df_list = []
    
    # Loop through each ward to find and read the corresponding file
    for ward in wards:
        file_name = setup.get_file_path(f'data/{file_prefix}_{ward}.csv')
        
        # Check if the file exists
        if os.path.exists(file_name):
            df = pd.read_csv(file_name)  
            df['ward'] = ward  # Add a column to indicate the ward
            df_list.append(df)  
        else:
            print(f"Warning: File {file_name} does not exist and will be skipped.")  

    if df_list:
        concatenated_df = pd.concat(df_list, ignore_index=True)
        
        # Reorder columns to place 'ward' as the first column
        columns = ['ward'] + [col for col in concatenated_df.columns if col != 'ward']
        concatenated_df = concatenated_df[columns]
        
        return concatenated_df  
    else:
        print("No files to concatenate.")  
        return pd.DataFrame()  # Return an empty df if no files were found
    
df_clients = concatenate_files(wards, 'gcai_client_profiles')
df_scenarios = concatenate_files(wards, 'gcai_client_scenarios')
df_records = concatenate_files(wards, 'gcai_client_notes')
df_summaries = concatenate_files(wards, 'gcai_client_summaries')

In [ ]:
# Function to process client data: Assigns unique ID to clients and merges with scenario data
def process_clients(df_clients, df_scenarios):
    df_clients = (df_clients
                  .assign(ct_id=df_clients.index + 1)
                  .merge(df_scenarios[['ward', 'client_id', 'complications', 'num_months']].drop_duplicates(),
                         on=['ward', 'client_id'], how='left'))
    return df_clients

df_clients = process_clients(df_clients, df_scenarios)

In [ ]:
# Function to process scenarios data
def process_scenarios(df_scenarios, df_clients):
    df_scenarios = (df_scenarios
                    # Complications and num_months have been added to the clients df
                    .drop(columns=['complications', 'num_months'])
                    # Get the unique client ID
                    .merge(df_clients[['ward', 'client_id', 'ct_id']], on=['ward', 'client_id'], how='left')
                    # Extract the month number from the string
                    .assign(month=lambda df: df['month'].str.extract(r'(\d+)').astype(int))
                    .rename(columns={'journey': 'scenario'})
                    [['ct_id', 'month', 'scenario']])
    return df_scenarios

df_scenarios = process_scenarios(df_scenarios, df_clients)

In [ ]:
def process_records(df_records, df_clients):
    # Generate a random start date for each ct_id
    unique_ct_ids = df_clients['ct_id'].unique()
    start_dates = {ct_id: datetime(2022, 1, 1) + timedelta(days=random.randint(0, 365)) for ct_id in unique_ct_ids}

    # Merge df_records with df_clients to add 'ct_id' to df_records
    df_records = df_records.merge(df_clients[['ward', 'client_id', 'ct_id']], on=['ward', 'client_id'], how='left')

    # Calculate the maximum 'dag' (day) value for each 'ct_id'
    max_days_per_ct_id = df_records.groupby('ct_id')['dag'].max().to_dict()

    # Function to calculate the date using the start_date for each ct_id and max 'dag' value
    def calculate_datetime(row, max_days_per_ct_id):
        max_day = max_days_per_ct_id.get(row['ct_id'], 15)  # Use the max 'dag' value or default to 15
        start_date = start_dates[row['ct_id']]  # Get the start date for the specific ct_id
        # Combine date and time
        base_date = start_date + timedelta(days=(row['month'] - 1) * max_day + (row['dag'] - 1))
        # Keep only digits in the time string
        time_digits = ''.join(filter(str.isdigit, row['tijd']))
        # Convert the cleaned time string to a time object using the '%H%M' format
        time_delta = pd.to_datetime(time_digits, format='%H%M').time()
        return datetime.combine(base_date, time_delta)

    # Apply the datetime calculation to each record and update the DataFrame
    df_records = (df_records
                  .assign(datetime=lambda df: df.apply(
                      lambda row: calculate_datetime(row, max_days_per_ct_id), axis=1))
                  .rename(columns={'rapportage': 'note'})
                  [['ct_id', 'datetime', 'note']])
    
    return df_records

df_records = process_records(df_records, df_clients)

In [ ]:
# Function to process summaries data
def process_summaries(df_summaries, df_clients):
    summaries_list = df_summaries['summary'].tolist()
    dict_list = [ast.literal_eval(s) for s in summaries_list]
    df_parsed_summaries = pd.DataFrame(dict_list).drop(columns=['client_id'])

    df_summaries = (df_summaries
                    .merge(df_clients[['ward', 'client_id', 'ct_id']], on=['ward', 'client_id'], how='left')
                    [['ct_id']])

    df_summaries = pd.concat([df_summaries, df_parsed_summaries], axis=1)
    return df_summaries

df_summaries = process_summaries(df_summaries, df_clients)


In [ ]:
def rename_client_columns(df_clients):
    # Rename and reorder columns for df_clients. 
    df_clients = (df_clients
                  .rename(columns={
                      'naam': 'name',
                      'type_dementie': 'dementia_type',
                      'somatiek': 'physical',
                      'adl': 'adl',
                      'mobiliteit': 'mobility',
                      'gedrag': 'behavior',
                  })
                  [['ct_id', 'ward', 'name', 'dementia_type', 'physical', 'adl',
                    'mobility', 'behavior', 'complications', 'num_months']])
    return df_clients

df_clients = rename_client_columns(df_clients)


In [ ]:
# Save final processed dataframes
df_clients.to_csv(FN_CLIENTS_DF, index=False)
df_scenarios.to_csv(FN_SCENARIOS_DF, index=False)
df_records.to_csv(FN_RECORDS_DF, index=False)
df_summaries.to_csv(FN_SUMMARIES_DF, index=False)


In [ ]:
# # Function to convert DataFrames to Hugging Face Datasets and push to hub
# def push_dataset_to_hub(df, dataset_name, hf_repo_name):
#     dataset = Dataset.from_pandas(df)
#     dataset.push_to_hub(f"{hf_repo_name}_{dataset_name}", private=True)

# push_dataset_to_hub(df_summaries, "summaries", hf_repo_name)
# push_dataset_to_hub(df_records, "records", hf_repo_name)
# push_dataset_to_hub(df_scenarios, "scenarios", hf_repo_name)
# push_dataset_to_hub(df_clients, "clients", hf_repo_name)